In [54]:
#pip install tqdm

In [20]:
import librosa
import h5py
import librosa.display as display
import numpy as np
import os
import IPython.display as ipd
from tqdm import tqdm

In [35]:
PCEN = True # True or False

In [22]:
base_dir = '/scratch/richardso21/DCASE_3_Models/grill_pcen'
os.chdir(base_dir)

In [34]:
def wav_to_h5(input_wav_dir):
    a,sr = librosa.load(input_wav_dir)
    
    a_log = librosa.feature.melspectrogram(a,sr=sr,n_fft=1024,hop_length=315,n_mels=80,fmax=11000)
    if PCEN == True:
        a_out = librosa.pcen(a_log*(2**31))
    elif PCEN == False:
        a_out = librosa.power_to_db(a_log,ref=np.max)
    
    duration = librosa.get_duration(a)
    frames = a_out.shape[1]
    timeframe = np.linspace(0,duration,num=frames)
    
    dir_name = os.path.basename(os.path.dirname(input_wav_dir))
    name = os.path.basename(input_wav_dir)
    
    with h5py.File('workingfiles/spect/{0}/{1}.h5'.format(dir_name,name), 'w') as data_file:
        data_file.create_dataset('features',data=a_out.T,dtype='float32')
        data_file.create_dataset('times',data=timeframe,dtype='float32')
    #print('Done creating {0}'.format(name))

In [ ]:
for folder in os.listdir('audio'):
    filelist_path = os.path.join('audio',folder)
    filelist = sorted(os.listdir(filelist_path))
    
    try:
        os.mkdir(os.path.join('workingfiles/spect',folder))
    except OSError as error:
        print(error)
        
    for file in tqdm(filelist,desc=folder):
        file_path = os.path.join('workingfiles/spect',folder,file)
        exist_counter = 0
        
        if os.path.exists('{0}.h5'.format(file_path)) == False:
            #wav_to_h5(f'{filelist_path}/{file}')
            wav_to_h5(os.path.join(filelist_path,file))
        else:
            break

In [9]:
'''#master_list = []
for folder in os.listdir('audio'):
    filelist = [sorted(os.listdir('audio/' + folder))]
    
    try:
        os.mkdir('workingfiles/spect/{0}'.format(folder))
    except OSError as error:
        print(error)
    
    for file in tqdm(filelist,desc='{0}'.format(folder)):
        wav_to_h5('audio/{0}/{1}'.format(folder,file), folder, file)
    
    #master_list.append(filelist) #FINAL'''

"master_list = []\nfor folder in os.listdir('audio'):\n    filelist = [sorted(i for i in os.listdir('audio/' + folder))]\n    os.mkdir('workingfiles/spect/{0}'.format(folder))\n    \n    for file in tqdm(filelist,desc='{0}'.format(folder)):\n        wav_to_h5('audio/{0}/{1}'.format(folder,file), folder, file)\n    \n    master_list.append(filelist) #FINAL"

In [1]:
'''try:
    os.mkdir('workingfiles/spect')
    os.mkdir('workingfiles/spect/{0}'.format('testing'))
except OSError as error:
    print(error)
    
folder = 'testing'
for file in tqdm(os.listdir('audio/testing/'),desc='{0}'.format('testing')):
    wav_to_h5('audio/{0}/{1}'.format(folder,file),folder,file)'''

"try:\n    os.mkdir('workingfiles/spect')\n    os.mkdir('workingfiles/spect/{0}'.format('testing'))\nexcept OSError as error:\n    print(error)\n    \nfolder = 'testing'\nfor file in tqdm(os.listdir('audio/testing/'),desc='{0}'.format('testing')):\n    wav_to_h5('audio/{0}/{1}'.format(folder,file),folder,file)"